In [1]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmocean as cmo
import os,sys,fnmatch,time
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib import animation
from salishsea_tools.nc_tools import scDataset
from salishsea_tools import (psu_tools, teos_tools)

from matplotlib import colors
from matplotlib import ticker
from matplotlib.colors import LinearSegmentedColormap
import gsw

%matplotlib inline

import sys
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/general_circulation')
import general_functions
import quicklook

import warnings
warnings.filterwarnings("ignore")

import time
from datetime import datetime 

/home/imachuca/anaconda3/envs/gsw/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


# Functions

Time elapsed (hh:mm:ss.ms) 0:05:25.748113

Okay, so let's look at level=0. At $e3t_0$, there are 8.75 m over it, and the density is $\rho_0$. 

At level=1, at $e3t_1$ there is an additional 8.75 m over it for a total of 17.5 m and the density is $\rho_1$.

At level=bottom, at $e3t_{bottom}$ there is a total of 1342.56 m over it and the density is $\rho_{bottom}$.

density ~ 1017 - 1028

http://www.oc.nps.edu/nom/day1/parta.html

http://www.ccpo.odu.edu/~atkinson/OEAS405/Chapter2_Stratified_Ocean/Lec_04_DensityEOS.pdf

In [2]:
def get_vars_pressure(dirname, fname, meshmaskname, time_ind):
    
    filesU = general_functions.get_files(dirname, fname, 'grid_U') 
    filesT = general_functions.get_files(dirname, fname, 'grid_T')
    
    y,x = slice(1,-1,None), slice(1,-1,None)

    dep_end=20 # to make it faster for now
    
    with scDataset(filesU) as dsU, scDataset(filesT) as dsT:
        vosaline0 = dsT.variables['vosaline'][:time_ind, :dep_end, y, x]#
        votemper0 = dsT.variables['votemper'][:time_ind, :dep_end, y, x]#
        sossheig0 = dsT.variables['sossheig'][:time_ind, y, x]
        deptht = dsT.variables['deptht'][:]
        sozotaux = dsU.variables['sozotaux'][:time_ind, 0, 0]

    with nc.Dataset(os.path.join(dirname, meshmaskname), 'r') as dsM:
        tmask0 = dsM.variables['tmask'][0, :dep_end, y, x]#
        tmask_ss0 = dsM.variables['tmask'][0, 0, y, x]
        e3t = dsM.variables['e3t_0'][0, :dep_end, 0, 0]#
    
    tmask = np.tile(tmask0, (len(sozotaux), 1, 1, 1)) 
    tmask_ss = np.tile(tmask_ss0, (len(sozotaux), 1, 1)) 

    vosaline = np.ma.array(vosaline0, mask=1 - tmask)
    votemper = np.ma.array(votemper0, mask=1 - tmask)
    sossheig = np.ma.array(sossheig0, mask=1 - tmask_ss)
    
    return vosaline, votemper, sossheig, tmask, e3t, deptht

In [3]:
def calculate_pressure(vosaline, votemper, sossheig, e3t):
    ''' returns pressure in decibars
    '''
    g = 9.81

    rho = np.full([vosaline.shape[-4], vosaline.shape[-3], vosaline.shape[-2], vosaline.shape[-1]], np.nan)
    p_stat0 = np.full_like(rho, np.nan)
    p_stat = np.full_like(rho, np.nan)
    p_surf = np.full([vosaline.shape[-4], vosaline.shape[-2], vosaline.shape[-1]], np.nan)
    pressure = np.full_like(rho, np.nan)

    for t in range(vosaline.shape[-4]):
        if t % 24 == 0:
            print(t)
        
        for k in range(vosaline.shape[-3]):
            gsw_vosaline = vosaline[t, k, :, :]
            gsw_votemper = votemper[t, k, :, :]
            rho[t, k, :, :] = gsw.rho(gsw_vosaline, gsw_votemper, 0)
            p_stat0[t, k, :, :] = rho[t, k, :, :] * e3t[k]
            
        for k in range(1, len(e3t)+1): 
            p_stat[t, k-1, :, :] = g * np.sum(p_stat0[t, :k, :, :], axis=0)
            
        p_surf[t, :, :] = rho[t, 0, :, :] * g * sossheig[t, :, :]
        
        for k in range(vosaline.shape[-3]):
            pressure[t, k, :, :] = p_stat[t, k, :, :] + p_surf[t, :, :]
            
    pressure_db = pressure * 0.0001
        
    return rho, pressure_db

In [4]:
def plot_pressure_daily(pressure_db, tmask, deptht, dep_ind_slice, kind, case):
    cmap = cmo.cm.speed
    cmap.set_bad('silver')

    pressure_dep = pressure_db[:, dep_ind_slice, :, :]
    
    xs = np.arange(pressure_dep.shape[-1])
    ys = np.arange(pressure_dep.shape[-2])
    #vmin, vmax = int(np.nanmin(pressure_dep)), int(np.nanmax(pressure_dep))
    #levels=np.linspace(vmin, vmax, 10)

    fig, axes = plt.subplots(3, 3, figsize = (20, 21))
    for ax, n in zip(axes.flatten(), np.arange(9)):
        pressure_day = quicklook.get_1day_avg(pressure_dep, n, n+1)
        p = ax.pcolormesh(xs, ys, pressure_day, cmap=cmap)#, vmin=vmin, vmax=vmax)
        #cs = ax.contour(xs, ys, pressure_day, levels=levels, colors='k', alpha=0.8)
        #ax.clabel(cs, inline=1, fontsize=10, fmt ='%r', colors='k')
        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax.get_yticklabels(), visible=False)
        ax.tick_params(axis='both', which='both', length=0)
        ax.set_aspect(aspect='equal')
        ax.set_xlim([0, xs[-1]])
        ax.set_ylim([0, ys[-1]])
        ax.set_title('Day ' + str(n+1), fontsize=20)
        cbar = fig.colorbar(p, ax=ax, fraction=0.05, orientation='horizontal', pad=0.009)
        cbar.ax.tick_params(labelsize=13)
        tick_locator = ticker.MaxNLocator(nbins=8)
        cbar.locator = tick_locator
        cbar.update_ticks()

    fig.tight_layout(w_pad=1.2, h_pad=0.01)
    fig.suptitle(kind + ' - ' + case +': ' + 'daily pressure [dbar] at depth = '+
                str(int(deptht[dep_ind_slice]))+' m', fontsize=25)
    plt.subplots_adjust(top=0.96)

    return fig

$$sum(\rho * e3t) * g = [\frac{kg}{m^3} * \frac{m}{1} * \frac{m}{s^2}] > [\frac{kg}{m^3} * \frac{m^2}{s^2}] > [\frac{kg}{m} * \frac{1}{s^2}] > [\frac{kg}{ms^2}] $$ 

$$ 1 Pa = 1 \frac{kg}{ms^2}$$

$$1 pascal [Pa] = 0.0001 decibar [dbar]$$

# Constants

In [5]:
dep_ind_slices = [1, 9, 16]
fname = '1_MCKNZ_1h_20170101_201701*'
meshmaskname = '1_mesh_mask.nc'
time_ind = 24 * 10

# Ideal

In [6]:
kind = 'ideal'
case = 'base' 
dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'

In [7]:
for dep_ind_slice in dep_ind_slices:
    vosaline, votemper, sossheig, tmask, e3t, deptht = get_vars_pressure(dirname, fname, meshmaskname, time_ind)

    rho, pressure_db = calculate_pressure(vosaline, votemper, sossheig, e3t)

    print(np.nanmin(rho), np.nanmax(rho))

    fig = plot_pressure_daily(pressure_db, tmask, deptht, dep_ind_slice, kind, case)
    
    fig.savefig('../writing_images/pressure_'+kind+'_'+case+'_'+str(dep_ind_slice)+'_daily.png', dpi=100, bbox_inches='tight')
    plt.close(fig) 

0
24
48
72
96
120
144
168
192
216
1017.6764579222775 1027.892093421783
0
24
48
72
96
120
144
168
192
216
1017.6764579222775 1027.892093421783
0
24
48
72
96
120
144
168
192
216
1017.6764579222775 1027.892093421783


# Real

In [8]:
kind = 'real'
case = 'base' 
dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'

In [9]:
for dep_ind_slice in dep_ind_slices:
    vosaline, votemper, sossheig, tmask, e3t, deptht = get_vars_pressure(dirname, fname, meshmaskname, time_ind)

    rho, pressure_db = calculate_pressure(vosaline, votemper, sossheig, e3t)

    print(np.nanmin(rho), np.nanmax(rho))

    fig = plot_pressure_daily(pressure_db, tmask, deptht, dep_ind_slice, kind, case)
    
    fig.savefig('../writing_images/pressure_'+kind+'_'+case+'_'+str(dep_ind_slice)+'_daily.png', dpi=100, bbox_inches='tight')
    plt.close(fig) 

0
24
48
72
96
120
144
168
192
216
1017.6764473312587 1027.8962550728056
0
24
48
72
96
120
144
168
192
216
1017.6764473312587 1027.8962550728056
0
24
48
72
96
120
144
168
192
216
1017.6764473312587 1027.8962550728056
